<a href="https://colab.research.google.com/github/SaltyWriter38/Int2Group22/blob/main/BatchNorm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch  
import torch.nn as nn 
import torchvision
import torch.nn.functional as F
import torch.optim as optim 
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
#Hyper parameters 
Epoch_num = 100
batch_size =64
learning_rate = 0.01

width = 224
height = 224

device =  torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
training_transfrom = transforms.Compose([transforms.Resize((width,height)), transforms.RandomRotation(30),
                transforms.RandomVerticalFlip(),transforms.RandomHorizontalFlip(), 
                transforms.ToTensor() ,transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])])

training_data =  datasets.Flowers102(root='data', split='train', download= True, transform= training_transfrom)

training_loader = DataLoader(training_data, batch_size=batch_size, shuffle= True, num_workers=2)

val_transform = transforms.Compose([transforms.Resize((width,height)),transforms.ToTensor(), 
                                    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])])
val_data = datasets.Flowers102(root='data', split = 'val', download=True, transform = val_transform)

val_loader = DataLoader(val_data, batch_size= batch_size, shuffle=True)

test_transform = transforms.Compose([transforms.Resize((width,height)), transforms.ToTensor(), 
                                     transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])])

test_data = datasets.Flowers102(root='data', split = 'test', download = True, transform = test_transform)
test_loader = DataLoader(test_data, batch_size= batch_size, shuffle = True)

100%|██████████| 344862509/344862509 [00:11<00:00, 30718862.13it/s]


Extracting data/flowers-102/102flowers.tgz to data/flowers-102


100%|██████████| 502/502 [00:00<00:00, 525465.59it/s]


100%|██████████| 14989/14989 [00:00<00:00, 13476617.93it/s]


In [4]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=96, kernel_size=(11,11), stride = (4,4), padding=(1,1))
        self.bn1 = nn.BatchNorm2d(96)
        self.conv2 = nn.Conv2d(in_channels=96, out_channels=256, kernel_size=(5,5), stride = (1,1), padding=(1,1))
        self.bn2 = nn.BatchNorm2d(256)
        self.conv3 = nn.Conv2d(in_channels=256, out_channels= 384, kernel_size= (3,3), stride=(1,1), padding=(1,1))
        self.bn3 = nn.BatchNorm2d(384)
        self.conv4 = nn.Conv2d(in_channels = 384, out_channels= 384, kernel_size=(3,3), stride=(1,1), padding=(1,1))
        self.bn4 = nn.BatchNorm2d(384)
        self.conv5 = nn.Conv2d(in_channels=384, out_channels= 256, kernel_size=(3,3), padding=(1,1))
        self.bn5 = nn.BatchNorm2d(256)
        self.pool = nn.MaxPool2d(kernel_size= (3,3), stride= (2,2))
        self.fc1 = nn.Linear(256*5*5, 4096)
        self.bn6 = nn.BatchNorm1d(4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.bn7 = nn.BatchNorm1d(4096)
        self.fc3 = nn.Linear(4096, 102)
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.pool(x)
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool(x)
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = F.relu(self.bn5(self.conv5(x)))
        x = self.pool(x)
        x = x.view(-1, 256 * 5 * 5)
        x = F.relu(self.bn6(self.fc1(x)))
        x = self.dropout(x)
        x = F.relu(self.bn7(self.fc2(x)))
        x = self.dropout(x)
        x = self.fc3(x)
        return x



model = CNN().to(device)

optimiser = optim.SGD(model.parameters(),lr=learning_rate, momentum=0.9)

criterion = nn.CrossEntropyLoss()

# scheduler = optim.lr_scheduler.StepLR(optimiser, step_size= 20, gamma=0.75, verbose= True)

# x = torch.randn(0, 3, 240, 240)
# y = model(x)
# print(y.shape)
print(model)

CNN(
  (conv1): Conv2d(3, 96, kernel_size=(11, 11), stride=(4, 4), padding=(1, 1))
  (bn1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(96, 256, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(256, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn3): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(384, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn4): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv5): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn5): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=6400, out_features=4096, bias=

In [5]:
image, label = next(iter(training_loader))

image = image.to(device)

x = F.relu(model.conv1(image))
x = model.pool(x)
# print(x.shape)
x = F.relu(model.conv2(x))
x = model.pool(x)
# print(x.shape)
x = F.relu(model.conv3(x))
# x = model.pool(x)
# print(x.shape)
x = F.relu(model.conv4(x))
# x = model.pool(x)
x = F.relu(model.conv5(x))
x = model.pool(x)
print(x.shape)

torch.Size([64, 256, 5, 5])


In [6]:
for epoch in range(Epoch_num):
    train_loss = 0.0
    valid_loss = 0.0
    correct = 0
    total = 0
    best_accuracy = 38

    
    model.train()
    for batch, (data,target) in enumerate(training_loader):
        
        data = data.to(device)
        target = target.to(device)
        
        optimiser.zero_grad()
        out = model(data)
        loss = criterion(out, target)
        loss.backward()
        optimiser.step()
        train_loss += loss.item() * data.size(0)
        
        
    model.eval()
    with torch.no_grad():
      for data, target in val_loader:
        
          data = data.to(device)
          target = target.to(device)
        
          out = model(data)

          _, predicted = torch.max(out.data, 1)

          total += target.size(0)
          correct += (predicted == target).sum().item()
          loss = criterion(out, target)

          valid_loss += loss.item() * data.size(0)
          
    accuracy = (correct / total) *100
    
    if(accuracy > best_accuracy):
      best_accuracy = accuracy
      torch.save(model, 'model_best')
      
    # if(learning_rate > 0.001):
    #   scheduler.step()
    train_loss = train_loss/len(training_loader.sampler)
    valid_loss = valid_loss/len(val_loader.sampler)
    # train_losses.append(train_loss)
    # valid_lossess.append(valid_loss)
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \tAccuracy: {:.6f}'.format(epoch, train_loss, valid_loss, accuracy))
    

def final_test(test_loader, model):
  with torch.no_grad():
    correct = 0 
    total = 0
    for data, target in test_loader:
      data = data.to(device)
      target = target.to(device)

      out = model(data)

      _, predicted = torch.max(out.data, 1)

      total += target.size(0)

      correct +=( predicted == target).sum().item()

    accuracy = (correct / total) *100
    print("Accuracy: {:.2f}".format(accuracy))

best_model = torch.load('model_best')
final_test(test_loader, best_model)

Epoch: 0 	Training Loss: 4.722095 	Validation Loss: 12.792345 	Accuracy: 0.980392
Epoch: 1 	Training Loss: 4.244541 	Validation Loss: 4.969439 	Accuracy: 7.352941
Epoch: 2 	Training Loss: 3.971824 	Validation Loss: 4.069124 	Accuracy: 9.509804
Epoch: 3 	Training Loss: 3.741353 	Validation Loss: 4.586453 	Accuracy: 10.000000
Epoch: 4 	Training Loss: 3.677025 	Validation Loss: 4.156301 	Accuracy: 14.901961
Epoch: 5 	Training Loss: 3.343192 	Validation Loss: 4.048090 	Accuracy: 16.470588
Epoch: 6 	Training Loss: 3.282412 	Validation Loss: 3.998925 	Accuracy: 16.862745
Epoch: 7 	Training Loss: 3.156085 	Validation Loss: 3.991497 	Accuracy: 14.705882
Epoch: 8 	Training Loss: 3.028406 	Validation Loss: 4.046847 	Accuracy: 19.117647
Epoch: 9 	Training Loss: 2.945892 	Validation Loss: 4.022876 	Accuracy: 20.882353
Epoch: 10 	Training Loss: 2.807807 	Validation Loss: 4.798572 	Accuracy: 16.372549
Epoch: 11 	Training Loss: 2.719064 	Validation Loss: 3.959620 	Accuracy: 22.647059
Epoch: 12 	Train